In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
import os

# Lấy đường dẫn hiện tại
current_path = os.getcwd()
print("Đường dẫn hiện tại là:", current_path)
print(current_path)
#
current_path = os.path.isfile(r"blackhole.csv")
print("Đường dẫn hiện tại là:", current_path)

Đường dẫn hiện tại là: c:\Users\Admin\Desktop\LVTN_Master\Transformer-based-6LowPan-Routing-Attack-Detection
c:\Users\Admin\Desktop\LVTN_Master\Transformer-based-6LowPan-Routing-Attack-Detection
Đường dẫn hiện tại là: True


In [4]:
file_paths = [ 
            r"blackhole.csv",
            r"dodag.csv",
            r"flooding.csv",
            r"rank.csv"
]

In [5]:
# Load all CSV files into a list of DataFrames
dfs = [pd.read_csv(file) for file in file_paths]

In [6]:
# Concatenate all DataFrames into a single DataFrame
data = pd.concat(dfs,ignore_index=True)

In [7]:
# Display the first few rows
print("Dataset Preview:")
print(data.head())

Dataset Preview:
    time  source  destination  length  info  transmission_rate_per_1000_ms  \
0  0.037      39         9999     0.0   1.0                       0.000000   
1  0.037      39         9999     0.0   1.0                       0.000000   
2  0.038      39         9999     0.0   1.0                       0.671176   
3  0.045      39         9999     0.0   1.0                       0.000000   
4  0.046      39         9999     0.0   1.0                       0.000000   

   reception_rate_per_1000_ms  transmission_average_per_sec  \
0                    0.671176                      0.000000   
1                    0.649873                      0.000000   
2                    0.652361                      0.462516   
3                    0.633786                      0.000000   
4                    0.630378                      0.000000   

   reception_average_per_sec  transmission_count_per_sec  \
0                   0.499879                    0.000000   
1              

In [8]:
# Basic dataset information
print("\nDataset Info:")
print(data.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1639975 entries, 0 to 1639974
Data columns (total 18 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   time                                 1639975 non-null  float64
 1   source                               1639975 non-null  int64  
 2   destination                          1639975 non-null  int64  
 3   length                               1639975 non-null  float64
 4   info                                 1639975 non-null  float64
 5   transmission_rate_per_1000_ms        1639975 non-null  float64
 6   reception_rate_per_1000_ms           1639975 non-null  float64
 7   transmission_average_per_sec         1639975 non-null  float64
 8   reception_average_per_sec            1639975 non-null  float64
 9   transmission_count_per_sec           1639975 non-null  float64
 10  reception_count_per_sec              1639975 non-nu

## Processing Features

In [9]:
"""
    'time' feature is a numerical feature
"""
# Processing Time to second unit

print(f"Number of elements feature: {len(data['time'].sort_values().unique())}\n")
print(data['time'].sort_values().unique())

Number of elements feature: 725703

[0.000000e+00 8.000000e-03 9.000000e-03 ... 4.570607e+03 4.670676e+03
 4.819550e+03]


In [10]:
"""
    'info' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['info'].sort_values().unique())}\n")
print(data['info'].sort_values().unique())

Number of elements feature: 10

[0.         0.41111834 0.52029776 0.55440486 0.57018941 0.57241282
 0.57289431 0.58018146 0.58259798 1.        ]


In [11]:
"""
    'length' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['length'].sort_values().unique())}\n")
print(data['length'].sort_values().unique())



Number of elements feature: 13

[0.         0.46394382 0.483634   0.48436797 0.48546724 0.48595518
 0.4943263  0.50241319 0.58086668 0.58814455 0.59556985 0.6370377
 1.        ]


In [12]:
"""
    'source' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['source'].sort_values().unique())}\n")
print(data['source'].sort_values().unique())

Number of elements feature: 101

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101]


In [13]:
"""
    'destination' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['destination'].sort_values().unique())}\n")
print(data['destination'].sort_values().unique())


Number of elements feature: 101

[   1    2    3    4    5    6    7    8    9   10   11   12   13   14
   15   16   17   18   19   20   21   22   23   24   25   26   27   28
   29   30   31   32   33   34   35   36   37   38   39   40   41   42
   43   44   45   46   47   48   49   50   51   52   53   54   55   56
   57   58   59   60   61   62   63   64   65   66   67   68   69   70
   71   72   73   74   75   76   77   78   79   80   81   82   83   84
   85   86   87   88   89   90   91   92   93   94   96   97   98   99
  100  101 9999]


In [14]:
"""
    'dis' feature is a numerical feature
"""

print(f"Number of elements feature: {len(data['dis'].sort_values().unique())}\n")
print(data['dis'].sort_values().unique())


Number of elements feature: 206

[0.         0.54036485 0.54062852 0.54855638 0.55271419 0.55327591
 0.55383946 0.55697299 0.55769355 0.55885325 0.56002156 0.56119881
 0.56179088 0.56373155 0.56509042 0.5672305  0.56800289 0.5689356
 0.56987484 0.57209284 0.57289431 0.57321631 0.57681449 0.57748026
 0.57781454 0.5791612  0.57950029 0.58052356 0.58155603 0.58242365
 0.58277259 0.58542524 0.5859636  0.58668562 0.58741257 0.58759509
 0.5905596  0.59188396 0.59226556 0.59245691 0.59322598 0.59556985
 0.59756734 0.59899101 0.59940178 0.60002135 0.6002288  0.60254228
 0.60382957 0.60448024 0.60601739 0.6062392  0.60668453 0.60690805
 0.60758211 0.60803444 0.60917585 0.60963675 0.61103479 0.61150605
 0.61341846 0.61366068 0.6141473  0.6151294  0.61562499 0.61587394
 0.61789413 0.61815033 0.62049493 0.62129258 0.62156033 0.62291343
 0.62346154 0.62401369 0.62683788 0.62741601 0.62799878 0.63037826
 0.63068135 0.63098575 0.63252782 0.63315429 0.63378647 0.63506847
 0.63539274 0.63571856 0.63604

In [15]:
"""
    'dao' feature is a numerical feature
"""

print(f"Number of elements feature: {len(data['dao'].sort_values().unique())}")
print(f"Pre-processed data: {data['dao'].sort_values().unique()}\n")


Number of elements feature: 233
Pre-processed data: [0.         0.48619902 0.49601751 0.49939676 0.50748767 0.50906192
 0.51112428 0.51624361 0.51697837 0.51881972 0.52153311 0.52389028
 0.52463758 0.52501178 0.52827146 0.53042026 0.53130943 0.53220128
 0.53425039 0.53579744 0.53644477 0.53670416 0.54023313 0.54062852
 0.54115677 0.54155378 0.5430156  0.54435321 0.54489065 0.54583456
 0.54596977 0.54637593 0.5465115  0.54965597 0.55117861 0.55187496
 0.55201456 0.55412193 0.55440486 0.55497215 0.55711685 0.55754918
 0.55812745 0.55914451 0.56119881 0.56313169 0.56328143 0.56343131
 0.56403242 0.56418308 0.56831308 0.56862398 0.5689356  0.5694044
 0.5708208  0.57113764 0.57193315 0.57370087 0.57451265 0.57483885
 0.57582267 0.57781454 0.57798204 0.57831774 0.58035239 0.58069499
 0.58259798 0.58382599 0.58471157 0.58650466 0.58668562 0.58723037
 0.5903718  0.59131425 0.5915038  0.59188396 0.59207458 0.59226556
 0.59361276 0.59537237 0.59656339 0.59716449 0.59797192 0.60085396
 0.6010633 

In [16]:
"""
    'dio' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['dio'].sort_values().unique())}\n")
print(data['dio'].sort_values().unique())



Number of elements feature: 383

[0.         0.30352159 0.31626509 0.32192599 0.32882383 0.34092735
 0.34139932 0.34763883 0.35212623 0.3532958  0.36093386 0.36195576
 0.36901425 0.36992355 0.37052335 0.37486957 0.37708657 0.37735914
 0.38029452 0.3828695  0.38312241 0.38511725 0.38825731 0.38849395
 0.38966715 0.3937608  0.39486427 0.39724445 0.39956328 0.40100899
 0.40202808 0.4050214  0.40811604 0.4096282  0.40981565 0.41385635
 0.4140364  0.41475362 0.41809772 0.41913332 0.41981854 0.42201796
 0.42351702 0.42532436 0.42710569 0.42774727 0.43075205 0.43199711
 0.4327695  0.43671857 0.43791221 0.44158263 0.44230645 0.44259506
 0.44573666 0.44644254 0.44868234 0.44923792 0.45130652 0.45267312
 0.45537825 0.45578093 0.45631665 0.45910751 0.46173379 0.46251605
 0.46626331 0.46767109 0.46792629 0.47008667 0.4733622  0.47398847
 0.47411359 0.4743637  0.47883786 0.47957886 0.48142618 0.48375639
 0.48473461 0.48863277 0.4891186  0.48984682 0.4943263  0.49517209
 0.49553446 0.49891415 0.5012

In [17]:
"""
    'transmission_count_per_sec' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['transmission_count_per_sec'].sort_values().unique())}\n")
print(data['transmission_count_per_sec'].sort_values().unique())



Number of elements feature: 467

[0.         0.30352159 0.31626509 0.32192599 0.32823239 0.34045152
 0.3418675  0.34806061 0.35290836 0.36161686 0.36195576 0.36262846
 0.36931865 0.37052335 0.37082137 0.37515034 0.37763072 0.38029452
 0.38312241 0.38362582 0.38511725 0.38849395 0.38896521 0.38966715
 0.3937608  0.39420386 0.39530181 0.39724445 0.39997865 0.40100899
 0.40243266 0.4050214  0.40541359 0.40868575 0.40981565 0.41240491
 0.4140364  0.41457476 0.41475362 0.4179242  0.41913332 0.42032979
 0.42185023 0.42351702 0.42565013 0.42710569 0.42726639 0.42774727
 0.4312203  0.43215194 0.4327695  0.43671857 0.43701789 0.43776353
 0.44172766 0.44201732 0.44259506 0.44559514 0.44644254 0.44882139
 0.44909918 0.44951507 0.45130652 0.4525369  0.45551256 0.45578093
 0.45631665 0.45897541 0.46134171 0.46160316 0.46173379 0.46225557
 0.46600658 0.46741565 0.46779872 0.46996002 0.47084551 0.47373809
 0.47398847 0.47423866 0.47760004 0.47834316 0.47871424 0.47945545
 0.48142618 0.48387876 0.4848

##### 'node_id' feature trùng lặp với 'From' feature nên có thể loại bỏ feature này

In [18]:
"""
    'transmission_rate_per_1000_ms' feature is a categorical feature
"""

print(f"Number of elements feature: {len(data['transmission_rate_per_1000_ms'].sort_values().unique())}\n")
print(data['transmission_rate_per_1000_ms'].sort_values().unique())


Number of elements feature: 470

[0.         0.30448435 0.31626509 0.32192599 0.32882383 0.34092735
 0.34139932 0.34806061 0.35212623 0.3532958  0.35740874 0.36127624
 0.36195576 0.36901425 0.37022408 0.37458774 0.37708657 0.37763072
 0.38003227 0.3828695  0.38362582 0.38511725 0.38801999 0.38896521
 0.38966715 0.39353842 0.3937608  0.39530181 0.39745772 0.39956328
 0.40141795 0.40223059 0.40482472 0.40521769 0.4083063  0.4096282
 0.41000276 0.41295152 0.413676   0.41421615 0.41511045 0.41775041
 0.41947644 0.4199892  0.42185023 0.42301941 0.42368244 0.42694479
 0.42710569 0.42774727 0.42822634 0.43199711 0.4327695  0.43671857
 0.43776353 0.44172766 0.44201732 0.44259506 0.44445864 0.44658337
 0.44868234 0.44909918 0.44965349 0.45130652 0.45144362 0.4525369
 0.45537825 0.45564679 0.45591499 0.45645037 0.45884323 0.46107996
 0.46160316 0.46173379 0.46212523 0.46497722 0.46574961 0.46690409
 0.46957974 0.46970655 0.47084551 0.4709718  0.47373809 0.4738633
 0.47710388 0.47760004 0.4780956

In [19]:
"""
    'reception_total_duration_per_sec' feature is a categorical feature ?
"""

print(f"Number of elements feature: {len(data['reception_total_duration_per_sec'].sort_values().unique())}\n")
print(data['reception_total_duration_per_sec'].sort_values().unique())



Number of elements feature: 575

[0.         0.14850649 0.16743631 0.16922203 0.17277952 0.17569176
 0.17704595 0.18754215 0.18887949 0.18983902 0.1929355  0.19571112
 0.19963561 0.20064171 0.20138228 0.20233214 0.20239858 0.20258986
 0.20272719 0.202853   0.20541076 0.20561786 0.21369979 0.21463381
 0.21832143 0.2194941  0.22077624 0.22296913 0.22324877 0.22851262
 0.22868411 0.23008951 0.2331384  0.23578814 0.24068706 0.24191739
 0.24499329 0.24643579 0.24884184 0.24884184 0.24905259 0.25232582
 0.2553361  0.2555122  0.2564663  0.25845084 0.25866564 0.26022231
 0.26042297 0.2610472  0.26235129 0.26372914 0.26612245 0.26837911
 0.26845647 0.27028322 0.27051501 0.27254337 0.27405604 0.27447531
 0.2757479  0.27632031 0.27808647 0.27913027 0.27928723 0.27978081
 0.28140942 0.28297766 0.28449022 0.28595127 0.28626934 0.28656382
 0.28743027 0.28828242 0.28873329 0.28967978 0.29006056 0.29026869
 0.29134899 0.29180594 0.29230526 0.29260103 0.29381886 0.2950045
 0.29615978 0.2983858  0.29876

In [20]:
"""
    'reception_rate_per_1000_ms' feature is a categorical feature ?
"""

print(f"Number of elements feature: {len(data['reception_rate_per_1000_ms'].sort_values().unique())}\n")
print(data['reception_rate_per_1000_ms'].sort_values().unique())


Number of elements feature: 1248

[0.         0.202853   0.21463381 ... 0.78883048 0.797147   1.        ]


In [21]:
"""
    'reception_count_per_sec' feature is a categorical feature 
"""
print(f"Number of elements feature: {len(data['reception_count_per_sec'].sort_values().unique())}\n")
print(data['reception_count_per_sec'].sort_values().unique())



Number of elements feature: 1253

[0.         0.21463381 0.23008951 ... 0.78883048 0.797147   1.        ]


#### 'Sending time', 'Sending_rate', 'Received_Packets', 'Forward_Packets' , 'Drop_count', 'Energy_consumption, 'etx' và 'node_honest_level' được coi là các numerical feature trong tập huấn luyện

In [22]:
"""
    'transmission_total_duration_per_sec' feature is numerical feature
"""
print(f"Number of elements feature: {len(data['transmission_total_duration_per_sec'].sort_values().unique())}\n")
print(data['transmission_total_duration_per_sec'].sort_values().unique())



Number of elements feature: 1634

[0.         0.15642561 0.16249224 ... 0.75500671 0.75931294 1.        ]


In [23]:
"""
    'reception_average_per_sec' feature is numerical feature
"""

print(f"Number of elements feature: {len(data['reception_average_per_sec'].sort_values().unique())}\n")
print(data['reception_average_per_sec'].sort_values().unique())



Number of elements feature: 9369

[0.         0.25232582 0.2555122  ... 0.78536619 0.797147   1.        ]


In [24]:
"""
    'transmission_average_per_sec' feature is numerical feature
"""

print(f"Number of elements feature: {len(data['transmission_average_per_sec'].sort_values().unique())}\n")
print(data['transmission_average_per_sec'].sort_values().unique())


Number of elements feature: 21764

[0.         0.28192347 0.28245251 ... 0.78792013 0.797147   1.        ]


# 'node_type' là feature được đánh nhãn cho huấn luyện
##### Do Node gồm 4 loại Node, với mục đích xác định node tấn công do đó 3 loại child_node, edge_node, root_node có thể coi là các normal node (0), 
##### Ngược lại với Blackhole node thì có thể coi là attack node(1).

In [25]:
# """
#     'node_type' feature is numerical feature
# """

# print(f"Number of elements 'node_type' feature: {len(data['node_type'].sort_values().unique())}\n")
# print(f"Pre-procesed {data['node_type'].sort_values().unique()}")


# mapping = {
#     'CHILD_NODE': 0,
#     'EDGE_NODE': 0,
#     'ROOT_NODE': 0,
#     'Blackhole Attack': 1
# }

# data['node_type_binary'] = data['node_type'].map(mapping)
# print(f"Procesed {data['node_type_binary'].sort_values().unique()}")

# # Remove 'node_type' feature from dataset
# data = data.drop(columns='node_type')

In [26]:
# Split data
X = data.drop(columns=['label', 'category'])
y = data['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1311980 entries, 513145 to 250827
Data columns (total 16 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   time                                 1311980 non-null  float64
 1   source                               1311980 non-null  int64  
 2   destination                          1311980 non-null  int64  
 3   length                               1311980 non-null  float64
 4   info                                 1311980 non-null  float64
 5   transmission_rate_per_1000_ms        1311980 non-null  float64
 6   reception_rate_per_1000_ms           1311980 non-null  float64
 7   transmission_average_per_sec         1311980 non-null  float64
 8   reception_average_per_sec            1311980 non-null  float64
 9   transmission_count_per_sec           1311980 non-null  float64
 10  reception_count_per_sec              1311980 non-null  float64
 11 

In [28]:
unique_values = np.unique(y_test, return_counts=True)
unique_values

(array([0, 1]), array([212101, 115894]))

In [29]:
x_train = {
    col: (
        X_train[col].values.astype('<U32').reshape(-1, 1) if col in ['info', 'length']
        else X_train[col].values.reshape(-1, 1)
    )
    for col in X_train.columns
}

In [30]:
import numpy as np

def create_context_windows(X, window_size=21, pad_value = 0 , dtype = tf.float32):
    """
    X: (num_packets, num_features)
    y: (num_packets,)
    window_size: số gói trong context (ví dụ 21)
    """
    half_left = 10
    half_right = 10

    num_packets = len(X)
    X_windows = []

    for i in range(num_packets):
        # xác định index trái - phải
        left = max(0, i - half_left)
        right = min(num_packets, i + half_right + 1)

        # cắt đoạn context
        context = X[left:right]

        # pad nếu thiếu (ở đầu hoặc cuối)
        if len(context) < window_size:
            pad_len = window_size - len(context)
            pad = np.full((pad_len, X.shape[1]), pad_value)  # pad bằng zero
            if left == 0:
                context = np.vstack([pad, context])
            else:
                context = np.vstack([context, pad])

        X_windows.append(context)

    return tf.constant(np.array(X_windows), dtype=dtype)

In [31]:
for col in ['info', 'length']:
    x_train[col] = tf.constant(x_train[col].flatten(), dtype=tf.string)
    x_train[col] = tf.reshape(x_train[col], (-1, 1))

In [32]:
for col in X_train.columns:
    if col in ['info', 'length']:
        x_train[col] = create_context_windows(x_train[col], 21, 'PAD', dtype=tf.string)
    else:
        x_train[col] = create_context_windows(x_train[col], 21)

In [33]:
for col in x_train.keys():
    print(f"Key:{col} - dtype:{x_train[col].dtype}")
    print(f"Key:{col} - type:{type(x_train[col])}")


y_train.dtype

Key:time - dtype:<dtype: 'float32'>
Key:time - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:source - dtype:<dtype: 'float32'>
Key:source - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:destination - dtype:<dtype: 'float32'>
Key:destination - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:length - dtype:<dtype: 'string'>
Key:length - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:info - dtype:<dtype: 'string'>
Key:info - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:transmission_rate_per_1000_ms - dtype:<dtype: 'float32'>
Key:transmission_rate_per_1000_ms - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:reception_rate_per_1000_ms - dtype:<dtype: 'float32'>
Key:reception_rate_per_1000_ms - type:<class 'tensorflow.python.framework.ops.EagerTensor'>
Key:transmission_average_per_sec - dtype:<dtype: 'float32'>
Key:transmission_average_per_sec - type:<class 'tensorflow.python.framework.ops.Eage

dtype('int64')

In [34]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import metrics

#### Step 2: Create lookup table and layer embedding for categorical features

In [35]:
source_lookup = tf.keras.layers.IntegerLookup(mask_token= 0, oov_token= -1, output_mode='int')
destination_lookup = tf.keras.layers.IntegerLookup(mask_token= 0, oov_token= -1, output_mode='int')
info_lookup = tf.keras.layers.StringLookup(mask_token="[PAD]", oov_token="[UNK]", output_mode='int')
length_lookup = tf.keras.layers.StringLookup(mask_token="[PAD]", oov_token="[UNK]", output_mode='int')

source_lookup.adapt(data['source'])
destination_lookup.adapt(data['destination'])
info_lookup.adapt(data['info'].astype(str))
length_lookup.adapt(data['length'].astype(str))

In [36]:
seq_len = 21
layers_embedding_dim = 8


source_embedding_layer = layers.Embedding(input_dim=source_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
source_input = tf.keras.Input(shape=(seq_len,), dtype=tf.int64, name='source')
source_index = source_lookup(source_input)
source_embedding = source_embedding_layer(source_index)
source_embedding = layers.Reshape((seq_len, 1, layers_embedding_dim), name='Source_Reshape')(source_embedding)


destination_embedding_layer = layers.Embedding(input_dim=destination_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
destination_input = tf.keras.Input(shape=(seq_len,), dtype=tf.int64, name='destination')
destination_index = destination_lookup(destination_input)
destination_embedding = destination_embedding_layer(destination_index)
destination_embedding = layers.Reshape((seq_len, 1, layers_embedding_dim), name='Destination_Reshape')(destination_embedding)


info_embedding_layer = layers.Embedding(input_dim=info_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
info_input = tf.keras.Input(shape=(seq_len,), dtype=tf.string, name='info')
info_index = info_lookup(info_input)
info_embedding = info_embedding_layer(info_index)
info_embedding = layers.Reshape((seq_len, 1, layers_embedding_dim), name='Info_Reshape')(info_embedding)

length_embedding_layer = layers.Embedding(input_dim=length_lookup.vocabulary_size(), output_dim= layers_embedding_dim)
length_input = tf.keras.Input(shape=(seq_len,), dtype=tf.string, name='length')
length_index = length_lookup(length_input)
length_embedding = length_embedding_layer(length_index)
length_embedding = layers.Reshape((seq_len, 1, layers_embedding_dim), name='Length_Reshape')(length_embedding)

In [37]:
print(source_embedding.shape)
print(destination_embedding.shape)
print(info_embedding.shape)
print(length_embedding.shape)



(None, 21, 1, 8)
(None, 21, 1, 8)
(None, 21, 1, 8)
(None, 21, 1, 8)


In [38]:
def build_float_numerical_feature_pipeline(feature_name, numerical_vector_dim, data, seq_len=21):
    """
    Tạo pipeline xử lý feature numerical: 
    1. Normalization
    2. Dense
    3. Reshape

    Input: (seq_len, 1)
    Output: (seq_len, numerical_vector_dim)
    """
    # Normalization layer
    norm_layer = tf.keras.layers.Normalization(axis=-1)
    norm_layer.adapt(data[feature_name].values.reshape(-1, 1))

    # Input cho feature
    feature_input = tf.keras.Input(shape=(seq_len,), dtype=tf.float32, name=feature_name)
    # print(feature_input.shape)
    # Normalize
    norm = norm_layer(feature_input)
    # print(norm.shape)
    # Dense projection
    # norm = layers.Reshape((seq_len, 1), name=f"{feature_name}_reshape_dense")(norm)
    norm = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                         name=f"{feature_name}_expand")(norm)


    feature_vector = tf.keras.layers.TimeDistributed(
        layers.Dense(numerical_vector_dim),
        name=f"{feature_name}_dense_td"
    )(norm) 
    
    # feature_vector = layers.Dense(numerical_vector_dim)(norm)
    # print(feature_vector.shape)
    # Reshape  (seq_len, 1, numerical_vector_dim) to concat with features
    feature_vector = layers.Reshape((seq_len, 1, numerical_vector_dim),name=f"{feature_name}_reshape")(feature_vector)

    return feature_input, feature_vector

def build_int_numerical_feature_pipeline(feature_name, numerical_vector_dim, data, seq_len=21):
    """
    Tạo pipeline xử lý feature numerical:
    1. Normalization
    2. Dense
    3. Reshape
    """
    # Normalization layer
    norm_layer = tf.keras.layers.Normalization(axis=-1)
    norm_layer.adapt(data[feature_name].values.reshape(-1, 1))

    # Input cho feature
    feature_input = tf.keras.Input(shape=(seq_len,), dtype=tf.int64, name=feature_name)

    # Normalize
    norm = norm_layer(feature_input)

    # Dense projection
    # feature_vector = layers.Dense(numerical_vector_dim)(norm)
    # print(feature_vector.shape)

    # Dense projection
    # norm = layers.Reshape((seq_len, 1), name=f"{feature_name}_reshape_dense")(norm)

    norm = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                         name=f"{feature_name}_expand")(norm)
    # print(norm.shape)
    feature_vector = tf.keras.layers.TimeDistributed(
        layers.Dense(numerical_vector_dim),
        name=f"{feature_name}_dense_td"
    )(norm) 
    print(feature_vector.shape)
    # Reshape  (seq_len, 1, numerical_vector_dim) to concat with features
    feature_vector = layers.Reshape((seq_len, 1, numerical_vector_dim), name=f"{feature_name}_reshape")(feature_vector)

    return feature_input, feature_vector

In [39]:
# Numerical -> Vector
numerical_vector_dim = 8
seq_len = 21
time_input, time_vector = build_float_numerical_feature_pipeline('time', numerical_vector_dim, data, seq_len)
dis_input, dis_vector = build_float_numerical_feature_pipeline('dis', numerical_vector_dim, data, seq_len)
dao_input, dao_vector = build_float_numerical_feature_pipeline('dao', numerical_vector_dim, data, seq_len)
dio_input, dio_vector = build_float_numerical_feature_pipeline('dio', numerical_vector_dim, data, seq_len)
transmission_count_per_sec_input, transmission_count_per_sec_vector = build_float_numerical_feature_pipeline('transmission_count_per_sec', numerical_vector_dim, data, seq_len)
transmission_rate_per_1000_ms_input, transmission_rate_per_1000_ms_vector = build_float_numerical_feature_pipeline('transmission_rate_per_1000_ms', numerical_vector_dim, data, seq_len)
reception_total_duration_per_sec_input, reception_total_duration_per_sec_vector = build_float_numerical_feature_pipeline('reception_total_duration_per_sec', numerical_vector_dim, data, seq_len)
reception_rate_per_1000_ms_input, reception_rate_per_1000_ms_vector = build_float_numerical_feature_pipeline('reception_rate_per_1000_ms', numerical_vector_dim, data, seq_len)
reception_count_per_sec_input, reception_count_per_sec_vector = build_float_numerical_feature_pipeline('reception_count_per_sec', numerical_vector_dim, data, seq_len)
transmission_total_duration_per_sec_input, transmission_total_duration_per_sec_vector = build_float_numerical_feature_pipeline('transmission_total_duration_per_sec', numerical_vector_dim, data, seq_len)
reception_average_per_sec_input, reception_average_per_sec_vector = build_float_numerical_feature_pipeline('reception_average_per_sec', numerical_vector_dim, data, seq_len)
transmission_average_per_sec_input, transmission_average_per_sec_vector = build_float_numerical_feature_pipeline('transmission_average_per_sec', numerical_vector_dim, data, seq_len)

In [40]:
print(time_vector.shape)


(None, 21, 1, 8)


In [41]:
concatenated = layers.Concatenate(axis=2)([time_vector, 
                                           source_embedding, 
                                           destination_embedding, 
                                           length_embedding, 
                                           info_embedding,
                                           transmission_rate_per_1000_ms_vector,
                                           reception_rate_per_1000_ms_vector,
                                           transmission_average_per_sec_vector,
                                           reception_average_per_sec_vector,
                                           transmission_count_per_sec_vector,
                                           reception_count_per_sec_vector,
                                           transmission_total_duration_per_sec_vector,
                                           reception_total_duration_per_sec_vector,
                                           dao_vector,
                                           dis_vector,
                                           dio_vector])
print(concatenated.shape)

(None, 21, 16, 8)


In [42]:
class BiasedMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads=1, key_dim=4, target_idx=None, bias_strength=2.0, **kwargs):
        super().__init__(**kwargs)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.target_idx = target_idx
        self.bias_strength = bias_strength

    def call(self, query, value, training=False):
        batch_size = tf.shape(query)[0]
        seq_len = tf.shape(query)[1]

        # attention mask (batch, seq_len, seq_len)
        mask = tf.ones((batch_size, seq_len, seq_len))

        if self.target_idx is not None:
            # Ưu tiên dòng cột của target_idx
            scale = tf.ones((batch_size, seq_len, seq_len))
            # target_mask = tf.one_hot(self.target_idx, seq_len)  # (seq_len,)
            # target_mask = tf.reshape(target_mask, (1, 1, seq_len))  # (1,1,seq_len)
            # scale = scale + (self.bias_strength - 1) * target_mask  # tăng trọng số cho target
            # mask = mask * scale

            # --- Bias cho cột (key bias) ---
            col_mask = tf.one_hot(self.target_idx, seq_len)   # (seq_len,)
            col_mask = tf.reshape(col_mask, (1, 1, seq_len))  # (1,1,key_len)
            scale = scale + (self.bias_strength - 1) * col_mask

            # --- Bias cho hàng (query bias) ---
            row_mask = tf.one_hot(self.target_idx, seq_len)   # (seq_len,)
            row_mask = tf.reshape(row_mask, (1, seq_len, 1))  # (1, query_len, 1)
            scale = scale + (self.bias_strength - 1) * row_mask
            # Áp dụng lên mask
            mask = mask * scale

        return self.mha(query=query, value=value, attention_mask=mask, training=training)

In [43]:
# ===== Transformer block for features in a package =====
def transformer_feature_block(x, num_heads=2, key_dim=4, ff_dim=16, dropout=0.1):
    # Multi-head attention
    attn_output = tf.keras.layers.MultiHeadAttention(num_heads=num_heads,
                                                      key_dim=key_dim)(x, x)
    attn_output = tf.keras.layers.Dropout(dropout)(attn_output)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed-forward layer
    ffn = tf.keras.Sequential([
        tf.keras.layers.Dense(ff_dim, activation="relu"),
        tf.keras.layers.Dense(x.shape[-1]),
    ])
    ffn_output = ffn(out1)
    ffn_output = tf.keras.layers.Dropout(dropout)(ffn_output)
    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

In [44]:
def transformer_sequence_block(x, num_heads=2, key_dim=4, ff_dim=16, dropout=0.1, target_idx=None):
    # Multi-head attention with bias
    attn_output = BiasedMultiHeadAttention(num_heads=num_heads,
                                           key_dim=key_dim,
                                           target_idx=target_idx)(x, x)
    attn_output = tf.keras.layers.Dropout(dropout)(attn_output)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

    # Feed-forward layer
    ffn = tf.keras.Sequential([
        tf.keras.layers.Dense(ff_dim, activation="relu"),
        tf.keras.layers.Dense(x.shape[-1]),
    ])
    ffn_output = ffn(out1)
    ffn_output = tf.keras.layers.Dropout(dropout)(ffn_output)

    return tf.keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

In [45]:
# ===== Step 1: Feature-level Transformer (per packet) =====

def feature_encoder(packet_features):
    packets_out = []
    num_packets = packet_features.shape[1]
    # Dense để ánh xạ feature → embedding
    for i in range(num_packets):
        packet_i = packet_features[:, i, :, :]
        packet_i = transformer_feature_block(packet_i, num_heads=2, key_dim=4, ff_dim=16)
        packet_i = layers.GlobalAveragePooling1D()(packet_i)
        packet_i = layers.Reshape((1, -1), name=f"Phong_{i}")(packet_i)  # (None, 1, 32)
        packets_out.append(packet_i)

    packets_tensor = layers.Concatenate(axis=1)(packets_out)
    return packets_tensor

In [46]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model, **kwargs):
        super().__init__(**kwargs)
        self.pos_encoding = self._positional_encoding(max_len, d_model)

    def _positional_encoding(self, max_len, d_model):
        pos = np.arange(max_len)[:, np.newaxis]  # (max_len, 1)
        i = np.arange(d_model)[np.newaxis, :]    # (1, d_model)
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        angle_rads = pos * angle_rates

        # apply sin to even indices, cos to odd indices
        sines = np.sin(angle_rads[:, 0::2])
        cosines = np.cos(angle_rads[:, 1::2])

        pos_encoding = np.zeros((max_len, d_model))
        pos_encoding[:, 0::2] = sines
        pos_encoding[:, 1::2] = cosines

        return tf.cast(pos_encoding[np.newaxis, ...], dtype=tf.float32)  # (1, max_len, d_model)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]

In [47]:
# # ===== Step 2: Packet-level Transformer (trên chuỗi nhiều packet) =====
packets = feature_encoder(concatenated)
pos_layer = PositionalEncoding(max_len=seq_len, d_model=numerical_vector_dim)
packets_with_pos = pos_layer(packets)
x = transformer_sequence_block(packets_with_pos, num_heads=2, key_dim=4, ff_dim=16, dropout=0.1, target_idx=10)
x = tf.keras.layers.Flatten()(x)
output = layers.Dense(1, activation='sigmoid')(x)

In [85]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# =========================
# Learning rate Schedule: Warmup + Cosine Decay
# =========================
class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_lr, batch_size, reference_batch_size, total_steps, warmup_steps, min_lr=1e-6):
        super().__init__()
        self.target_lr = base_lr * (batch_size / reference_batch_size)
        self.base_lr = base_lr
        self.batch_size = batch_size
        self.reference_batch_size = reference_batch_size
        self.total_steps = total_steps
        self.warmup_steps = warmup_steps
        self.min_lr = min_lr

    def __call__(self, step):
        # Phase 1: warmup (tăng tuyến tính)
        if self.warmup_steps > 0:
            warmup_lr = self.base_lr * tf.cast(step, tf.float32) / tf.cast(self.warmup_steps, tf.float32)
        else:
            warmup_lr = self.base_lr

        # Phase 2: cosine decay
        cosine_decay = 0.5 * (1 + tf.cos(
            tf.constant(np.pi) * 
            (tf.cast(step - self.warmup_steps, tf.float32) / tf.cast(self.total_steps - self.warmup_steps, tf.float32))
        ))
        decayed = (self.base_lr - self.min_lr) * cosine_decay + self.min_lr

        return tf.cond(step < self.warmup_steps,
                       lambda: warmup_lr,
                       lambda: decayed)
    
    def get_config(self):
        return {
            "base_lr": self.base_lr,
            "batch_size": self.batch_size,
            "reference_batch_size": self.reference_batch_size,
            "total_steps": self.total_steps,
            "warmup_steps": self.warmup_steps,
            "min_lr": self.min_lr
        }

In [86]:
# =========================
# Setup Optimizer AdamW + LR schedule
# =========================
epochs = 100
batch_size = 1024
reference_batch_size = 32
steps_per_epoch = y_train.shape[0] // batch_size  # ví dụ, = len(train_data)//batch_size
total_steps = epochs * steps_per_epoch
warmup_steps = int(0.1 * total_steps)  # warmup 10%

base_lr = 1e-3
lr_schedule = WarmupCosineDecay(base_lr, batch_size, reference_batch_size, total_steps, warmup_steps, min_lr=1e-6)

optimizer = tf.keras.optimizers.AdamW(
    learning_rate=lr_schedule,
    weight_decay=1e-4
)

In [87]:
inputs=[    
    time_input, 
    source_input, 
    destination_input, 
    length_input, 
    info_input,
    transmission_rate_per_1000_ms_input,
    reception_rate_per_1000_ms_input,
    transmission_average_per_sec_input,
    reception_average_per_sec_input,
    transmission_count_per_sec_input,
    reception_count_per_sec_input,
    transmission_total_duration_per_sec_input,
    reception_total_duration_per_sec_input,
    dao_input,
    dis_input,
    dio_input]


model = tf.keras.Model(inputs=inputs, outputs=output)
model.compile(optimizer= optimizer, loss='binary_crossentropy', metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])


In [88]:
model.input

[<KerasTensor shape=(None, 21), dtype=float32, sparse=False, ragged=False, name=time>,
 <KerasTensor shape=(None, 21), dtype=int64, sparse=False, ragged=False, name=source>,
 <KerasTensor shape=(None, 21), dtype=int64, sparse=False, ragged=False, name=destination>,
 <KerasTensor shape=(None, 21), dtype=string, sparse=False, ragged=False, name=length>,
 <KerasTensor shape=(None, 21), dtype=string, sparse=False, ragged=False, name=info>,
 <KerasTensor shape=(None, 21), dtype=float32, sparse=False, ragged=False, name=transmission_rate_per_1000_ms>,
 <KerasTensor shape=(None, 21), dtype=float32, sparse=False, ragged=False, name=reception_rate_per_1000_ms>,
 <KerasTensor shape=(None, 21), dtype=float32, sparse=False, ragged=False, name=transmission_average_per_sec>,
 <KerasTensor shape=(None, 21), dtype=float32, sparse=False, ragged=False, name=reception_average_per_sec>,
 <KerasTensor shape=(None, 21), dtype=float32, sparse=False, ragged=False, name=transmission_count_per_sec>,
 <KerasTens

In [89]:
model.summary()

Model: "functional_26"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ time (InputLayer)   │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_rate_… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_rate_per… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_avera… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_average_… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_count… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_count_pe… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transmission_total… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reception_total_du… │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dao (InputLayer)    │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dis (InputLayer)    │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dio (InputLayer)    │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 21)        │          3 │ time[0][0]        │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ source (InputLayer) │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ destination         │ (None, 21)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ length (InputLayer) │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ info (InputLayer)   │ (None, 21)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_5     │ (None, 21)        │          3 │ transmission_rat… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_7     │ (None, 21)        │          3 │ reception_rate_p… │
│ (Normalization)     │                   │            │                 

 Total params: 15,461 (60.44 KB)

 Trainable params: 15,425 (60.25 KB)

 Non-trainable params: 36 (192.00 B)

In [90]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

# # Callback custom: dừng khi thấy overfitting rõ rệt
# class OverfitStop(Callback):
#     def __init__(self, patience=3, gap_threshold=0.05):
#         super(OverfitStop, self).__init__()
#         self.patience = patience
#         self.gap_threshold = gap_threshold
#         self.wait = 0

#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         train_loss = logs.get("loss")
#         val_loss = logs.get("val_loss")

#         if val_loss and train_loss and (val_loss - train_loss) > self.gap_threshold:
#             self.wait += 1
#             if self.wait >= self.patience:
#                 print(f"\n⚠️ Dừng sớm vì phát hiện overfitting tại epoch {epoch+1}")
#                 self.model.stop_training = True
#         else:
#             self.wait = 0


def get_callbacks(model_path="best_model.keras"):
    # 1. Lưu model tốt nhất
    checkpoint = ModelCheckpoint(
        model_path,
        monitor="val_loss",
        save_best_only=True,
        mode="min",
        verbose=1
    )

    # 2. EarlyStopping
    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=3,  # số epoch chờ trước khi dừng
        restore_best_weights=True,
        verbose=1
    )

    return [checkpoint, early_stop]

In [91]:
callbacks = get_callbacks("best_model.keras")

In [ ]:
history = model.fit(
    x_train, y_train,
    batch_size=1024,
    epochs=100,
    validation_split=0.2,   # lấy 20% dữ liệu train làm validation
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7856 - loss: 0.4672 - precision: 0.7144 - recall: 0.6535
Epoch 1: val_loss improved from inf to 0.36451, saving model to best_model.keras
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1907s 2s/step - accuracy: 0.7857 - loss: 0.4672 - precision: 0.7144 - recall: 0.6536 - val_accuracy: 0.8601 - val_loss: 0.3645 - val_precision: 0.7752 - val_recall: 0.8516
Epoch 2/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8635 - loss: 0.3525 - precision: 0.7905 - recall: 0.8349
Epoch 2: val_loss improved from 0.36451 to 0.26012, saving model to best_model.keras
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 1532s 1s/step - accuracy: 0.8636 - loss: 0.3525 - precision: 0.7905 - recall: 0.8349 - val_accuracy: 0.9068 - val_loss: 0.2601 - val_precision: 0.8450 - val_recall: 0.9023
Epoch 3/100
1025/1025 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8987 - loss: 0.2689 - precision: 0.8369 - recall: 0.8864
Epoch 3: val_loss improved from 0.26012 to 0.21014, s

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model("best_model.keras")

In [ ]:
x_test = {
    col: (
        X_test[col].values.astype('<U32').reshape(-1, 1) if col in ['info', 'length']
        else X_test[col].values.reshape(-1, 1)
    )
    for col in X_test.columns
}

In [ ]:
for col in ['info', 'length']:
    x_test[col] = tf.constant(x_test[col].flatten(), dtype=tf.string)
    x_test[col] = tf.reshape(x_test[col], (-1, 1))

In [ ]:
for col in X_test.columns:
    if col in ['info', 'length']:
        x_test[col] = create_context_windows(x_test[col], 21, 'PAD', dtype=tf.string)
    else:
        x_test[col] = create_context_windows(x_test[col], 21)

In [ ]:
model.evaluate(x_test, y_test)

10250/10250 ━━━━━━━━━━━━━━━━━━━━ 1122s 109ms/step - accuracy: 0.9928 - loss: 0.0174 - precision: 0.9910 - recall: 0.9885


[0.017870794981718063,
 0.9925944209098816,
 0.991060733795166,
 0.9878969788551331]